In [10]:
# A atividade deverá ser realizada em um novo arquivo do Google Colab.
# Com o arquivo csv que você pesquisou, execute as seguintes atividades:

# https://www.kaggle.com/datasets/nelgiriyewithana/top-spotify-songs-2023?resource=download

# 1 - Inicie o ambiente Spark através da biblioteca pyspark e crie o objeto spark para manipulação dos dados.

# 2 – Altere o nome de pelo menos duas colunas de seu Data Frame.

# 3 – Converta o tipo de dados de alguma coluna de sua escolha: Pode ser uma conversão para float, int ou date.

# 4 – Realize 3 consultas através da função pyspark.sql. Pelo menos uma das consultas deve conter alguma função de agregação: count(), sum(), avg(), etc.
# Descreva qual o tipo de análise você fará no Data Frame e depois exiba o código em Python com o resultado da análise.

# 5 – Com alguma das consultas da atividade 3 crie um novo DataFrame e salve-o em formato .csv e em formato .orc.

In [11]:
# 1. Iniciando ambiente spark
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

# Criando o objeto spark
spark = SparkSession.builder.appName("Most Streamed Spotify Songs 2023").getOrCreate()


In [18]:
# 2. Alterando o nome de duas colunas do Data Frame.

# Importando arquivos para o spark
url = "/content/drive/MyDrive/Curso SENAI/Arquivo para Avaliação/Most Streamed Spotify Songs 2023/spotify-2023.csv/"

musicas = spark.read.options(header=True).csv(
    url,
    sep=",",
    inferSchema=True
)

musicas = musicas.withColumnsRenamed({
    "track_name":"Titulo_Faixa",
    "artist(s)_name":"Nome_Artista",
    "released_year": "Ano_Lancamento"})

In [20]:
# 3. Converta o tipo de dados de alguma coluna de sua escolha: Pode ser uma conversão para float, int ou date.

musicas = musicas.withColumn("bpm", musicas.bpm.cast("float"))

musicas.printSchema()

root
 |-- Titulo_Faixa: string (nullable = true)
 |-- Nome_Artista: string (nullable = true)
 |-- artist_count: integer (nullable = true)
 |-- Ano_Lancamento: integer (nullable = true)
 |-- released_month: integer (nullable = true)
 |-- released_day: integer (nullable = true)
 |-- in_spotify_playlists: integer (nullable = true)
 |-- in_spotify_charts: integer (nullable = true)
 |-- streams: string (nullable = true)
 |-- in_apple_playlists: integer (nullable = true)
 |-- in_apple_charts: integer (nullable = true)
 |-- in_deezer_playlists: string (nullable = true)
 |-- in_deezer_charts: integer (nullable = true)
 |-- in_shazam_charts: string (nullable = true)
 |-- bpm: float (nullable = true)
 |-- key: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- danceability_%: integer (nullable = true)
 |-- valence_%: integer (nullable = true)
 |-- energy_%: integer (nullable = true)
 |-- acousticness_%: integer (nullable = true)
 |-- instrumentalness_%: integer (nullable = true)
 

In [21]:
# 4 – Realize 3 consultas através da função pyspark.sql. Pelo menos uma das consultas deve conter alguma função de agregação: count(), sum(), avg(), etc.
# Descreva qual o tipo de análise você fará no Data Frame e depois exiba o código em Python com o resultado da análise.

# Criando os views do DataFrame:
musicas.createOrReplaceTempView("V_musicas")

# Usando a função spark.sql
consulta_tabelas = spark.sql( """
 SELECT * FROM V_musicas
"""
).show(3, False)

# Contando a quatidade de artistas
consulta_um = spark.sql( """
 SELECT COUNT(*) Nome_Artista FROM V_musicas
"""
).show()

# Média de batida por minuto (bpm)
consulta_dois = spark.sql( """
 SELECT AVG(bpm) AS media FROM V_musicas
"""
).show()

# Musica mais reproduzida
colsulta_tres = spark.sql( """
  SELECT MAX(Streams) AS Streams, Titulo_Faixa FROM V_musicas GROUP BY Titulo_Faixa
 """
).show(3, False)


+-----------------------------------+----------------+------------+--------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+-----+---+-----+--------------+---------+--------+--------------+------------------+----------+-------------+
|Titulo_Faixa                       |Nome_Artista    |artist_count|Ano_Lancamento|released_month|released_day|in_spotify_playlists|in_spotify_charts|streams  |in_apple_playlists|in_apple_charts|in_deezer_playlists|in_deezer_charts|in_shazam_charts|bpm  |key|mode |danceability_%|valence_%|energy_%|acousticness_%|instrumentalness_%|liveness_%|speechiness_%|
+-----------------------------------+----------------+------------+--------------+--------------+------------+--------------------+-----------------+---------+------------------+---------------+-------------------+----------------+----------------+-----+---+-----+--------------+-----

In [22]:
# 5. Com alguma das consultas da atividade 3 crie um novo DataFrame e salve-o em formato .csv e em formato .orc.

# Criando novo data frame:
novoDf = spark.sql( """
    SELECT * FROM V_musicas
    """
)

# Salvando data frame em formato .orc:
url = "/content/drive/MyDrive/Curso SENAI/Arquivo para Avaliação/Most Streamed Spotify Songs 2023/Arquivo em orc"
novoDf.coalesce(1).write.orc(
    url,
    mode="overwrite"
)

# Salvando data frame em formato .orc:
url = "/content/drive/MyDrive/Curso SENAI/Arquivo para Avaliação/Most Streamed Spotify Songs 2023/Arquivo em csv"
novoDf.coalesce(1).write.csv(
    url,
    mode="overwrite"
)

